In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
pd.set_option('display.max_columns', 81)

Data yang digunakan adalah data yang telah dikoreksi baseline

In [15]:
A_csv_file = glob.glob("../data_baseline/_A*.csv") # Mencari semua berkas CSV yang dimulai dengan 'A' dalam direktori 'data'
B_csv_file = glob.glob("../data_baseline/_B*.csv") # Mencari semua berkas CSV yang dimulai dengan 'B' dalam direktori 'data'

In [16]:
def get_MaxGradient(dataframe, column):
    array = dataframe[column].to_numpy()
    return max(np.gradient(array, edge_order=2))

In [17]:
def extract_FeaturesCubic_2W(CSV_files, label, return_dataset=False):
    """extract max, mean, median, and gradients with two windows method from given CSV files

    Args:
        CSV_files (list): csv files
        label (str): label
        return_dataset (bool, optional): wheter get formated dataset. Defaults to False.

    Returns:
        DataFrame: if return_dataset is True
        array: if return_dataset is False
    """
    maxi1 = np.empty((0, 10))
    maxi2 = np.empty((0, 10))
    mean1 = np.empty((0, 10))
    mean2 = np.empty((0, 10))
    median1 = np.empty((0, 10))
    median2 = np.empty((0, 10))
    grad1 = np.empty((0, 10))
    grad2 = np.empty((0, 10))
    for csv_file in CSV_files:
        df = pd.read_csv(csv_file)

        first_window = df[(df['time(s)'] >= 10) & (df['time(s)'] <= 15)]
        second_window = df[(df['time(s)'] > 15) & (df['time(s)'] <= 30)]

        df.drop(columns=["time(s)","Temp","Humid"], inplace=True)

        _max_list1 = []
        _max_list2 = []
        _mean_list1 = []
        _mean_list2 = []
        _median_list1 = []
        _median_list2 = []
        _grad_list1 = []
        _grad_list2 = []

        for cols, _ in df.iteritems():        
            _max_list1.append(first_window[cols].max())
            _max_list2.append(second_window[cols].max())

            _mean_list1.append(first_window[cols].mean())
            _mean_list2.append(second_window[cols].mean())

            _median_list1.append(first_window[cols].median())
            _median_list2.append(second_window[cols].median())

            _grad_list1.append(get_MaxGradient(first_window, cols))
            _grad_list2.append(get_MaxGradient(second_window, cols))

        maxi1 = np.vstack((maxi1, np.array(_max_list1).T))
        maxi2 = np.vstack((maxi2, np.array(_max_list2).T))
        mean1 = np.vstack((mean1, np.array(_mean_list1).T))
        mean2 = np.vstack((mean2, np.array(_mean_list2).T))
        median1 = np.vstack((median1, np.array(_median_list1).T))
        median2 = np.vstack((median2, np.array(_median_list2).T))
        grad1 = np.vstack((grad1, np.array(_grad_list1).T))
        grad2 = np.vstack((grad2, np.array(_grad_list2).T))
    
    if return_dataset:
        mean_df_1 = pd.DataFrame(mean1, columns=[f"mu1{i+1}" for i in range(10)])
        mean_df_2 = pd.DataFrame(mean2, columns=[f"mu2{i+1}" for i in range(10)])
        max_df_1 = pd.DataFrame(maxi1, columns=[f"max1{i+1}" for i in range(10)])
        max_df_2 = pd.DataFrame(maxi2, columns=[f"max2{i+1}" for i in range(10)])
        median_df_1 = pd.DataFrame(median1, columns=[f"med1{i+1}" for i in range(10)])
        median_df_2 = pd.DataFrame(median2, columns=[f"med2{i+1}" for i in range(10)])
        grad_df_1 = pd.DataFrame(grad1, columns=[f"grad1{i+1}" for i in range(10)])
        grad_df_2 = pd.DataFrame(grad2, columns=[f"grad2{i+1}" for i in range(10)])
        dataset = pd.concat([mean_df_1, max_df_1, median_df_1, grad_df_1,
                             mean_df_2, max_df_2, median_df_2, grad_df_2], axis=1)
        dataset["label"] = label
        return dataset
        
    else:        
        all_arrays = [maxi1, maxi2, mean1, mean2, median1, median2, grad1,grad2]
        all_arrays = np.transpose(all_arrays, (1, 2, 0))
        return all_arrays

In [18]:
A_array = extract_FeaturesCubic_2W(A_csv_file, label="A")
B_array = extract_FeaturesCubic_2W(B_csv_file, label="B")
print(A_array.shape)
print(B_array.shape)

(35, 10, 8)
(35, 10, 8)


In [19]:
A_array_file_filename = 'A_array_2W.npy'
B_array_file_filename = 'B_array_2W.npy'

np.save(f"../Exp/{A_array_file_filename}", A_array)
np.save(f"../Exp/{B_array_file_filename}", B_array)

In [20]:
A_tensor =  tf.convert_to_tensor(A_array)
B_tensor =  tf.convert_to_tensor(B_array)
print(A_tensor.shape)
print(B_tensor.shape)

(35, 10, 8)
(35, 10, 8)


In [21]:
A_tensor_file_filename = 'A_tensor_2W.tf'
B_tensor_file_filename = 'B_tensor_2W.tf'

tf.io.write_file(f"../Exp/{A_tensor_file_filename}", tf.io.serialize_tensor(A_tensor))
tf.io.write_file(f"../Exp/{B_tensor_file_filename}", tf.io.serialize_tensor(B_tensor))

In [22]:
# how to load
# tensor_bytes = tf.io.read_file(file_path)

# tensor_tf = tf.io.parse_tensor(tensor_bytes, out_type=tf.int32)

###  Get Dataset

In [23]:
A_dataset = extract_FeaturesCubic_2W(A_csv_file, label="A", return_dataset=True)
B_dataset = extract_FeaturesCubic_2W(B_csv_file, label="B", return_dataset=True)

In [24]:
full_dataset = pd.concat([A_dataset, B_dataset], axis=0)
full_dataset.reset_index(drop=True, inplace=True)

In [25]:
full_dataset[full_dataset["label"]=="A"]

,mu11,mu12,mu13,mu14,mu15,mu16,mu17,mu18,mu19,mu110,max11,max12,max13,max14,max15,max16,max17,max18,max19,max110,med11,med12,med13,med14,med15,med16,med17,med18,med19,med110,grad11,grad12,grad13,grad14,grad15,grad16,grad17,grad18,grad19,grad110,mu21,mu22,mu23,mu24,mu25,mu26,mu27,mu28,mu29,mu210,max21,max22,max23,max24,max25,max26,max27,max28,max29,max210,med21,med22,med23,med24,med25,med26,med27,med28,med29,med210,grad21,grad22,grad23,grad24,grad25,grad26,grad27,grad28,grad29,grad210,label
0,131.579338,8.063088,-1.047255,525.389387,251.257108,190.941765,4.633137,800.540172,512.948284,964.983358,457.98375,27.46750,-0.28500,1454.92125,758.11250,533.78000,11.91500,2195.48625,1416.36250,2089.96375,67.98375,4.34250,-1.16000,309.29625,158.23750,136.28000,3.91500,431.98625,341.36250,968.96375,18.9375,1.3125,0.0625,63.8125,27.0625,18.5000,0.4375,107.3750,56.1875,105.3125,1143.098750,114.152500,1.995833,1584.830417,835.445000,560.770833,18.780000,2356.063750,1733.748333,2199.922917,1310.98375,170.34250,3.71500,1683.54625,1013.86250,666.03000,20.54000,2415.36125,1795.73750,2276.46375,1270.29625,122.84250,2.34000,1607.54625,866.73750,579.15500,19.79000,2370.36125,1755.23750,2219.27625,19.0000,1.4375,0.1250,17.1250,17.1875,9.5000,0.3125,18.8125,21.2500,13.0625,A
1,59.562721,3.265270,-0.124020,311.589755,105.164608,86.858039,1.998333,428.542819,266.773064,593.225735,208.88625,10.84125,0.30000,999.07750,327.39500,253.26000,5.41500,1525.12125,825.36375,1590.26250,31.88625,1.84125,-0.20000,161.32750,67.77000,60.88500,1.66500,169.99625,155.36375,397.76250,9.0000,0.5000,0.0625,38.5000,11.1875,8.3750,0.3125,63.8125,29.3125,65.5625,753.524583,50.902083,1.735833,1368.175000,490.268333,350.165000,9.446667,2226.197917,1355.010417,1911.405833,1000.63625,80.84125,2.80000,1454.20250,569.64500,393.88500,10.79000,2325.49625,1459.61375,1986.51250,838.13625,53.90375,1.92500,1393.70250,502.45750,357.13500,10.16500,2284.12125,1433.05125,1935.13750,10.0000,0.6250,0.1875,24.7500,10.0625,6.0625,0.1875,45.1875,21.5625,20.9375,A
2,78.161348,4.082672,0.090858,332.101740,125.539314,104.979804,2.434044,509.280025,294.737132,667.627255,264.79125,13.90375,0.32125,1051.00125,396.13000,308.72000,6.19875,1654.34375,895.28125,1659.49000,43.91625,2.02875,0.07125,167.50125,80.25500,73.47000,2.07375,227.59375,173.28125,519.36500,11.0625,0.6250,0.0625,41.1250,13.5625,10.1250,0.3750,68.3750,32.1875,71.8750,868.544583,66.476250,2.172917,1375.864583,598.110000,418.263333,11.142083,2139.526250,1370.835417,1929.052500,1100.79125,106.40375,3.57125,1453.50125,687.75500,467.47000,12.82375,2209.21875,1457.03125,1992.49000,972.10375,70.21625,2.38375,1397.87625,612.81750,426.65750,11.94875,2174.28125,1431.40625,1948.49000,12.0000,0.8125,0.1250,23.5625,12.4375,7.4375,0.1875,35.5000,21.5625,18.0625,A
3,221.202696,13.453676,-0.900564,688.458578,393.714020,278.216078,6.689142,1031.911201,671.839044,1172.535319,737.83750,47.13750,-0.11625,1681.76250,1093.66500,729.39500,16.70875,2446.78375,1659.10375,2277.73875,121.21250,7.13750,-0.99125,506.01250,262.16500,210.64500,5.83375,736.28375,544.72875,1373.23875,27.4375,2.0000,0.0625,84.0625,41.6250,27.1875,0.6250,143.3125,71.0000,126.1250,1379.495833,194.484167,3.806250,1735.615000,1075.934167,700.977500,27.191250,2495.337917,1837.380417,2315.066250,1498.33750,288.88750,6.50875,1823.76250,1283.79000,834.14500,29.95875,2542.40875,1884.10375,2373.61375,1453.65000,209.51250,4.32125,1757.38750,1124.60250,726.58250,28.95875,2506.90875,1849.22875,2331.80125,26.0000,2.3750,0.3750,11.9375,16.6250,9.6250,0.4375,9.5000,15.8125,8.0625,A
4,69.817475,3.064216,0.337623,332.132451,124.601373,104.529289,2.968333,462.369436,289.056618,660.983922,242.62875,10.77500,0.78125,1067.63000,380.42000,300.55625,7.26000,1603.15375,893.26250,1700.18000,38.25375,1.40000,0.28125,167.13000,84.04500,75.68125,2.76000,187.77875,168.13750,486.18000,10.4375,0.5625,0.0625,41.5000,12.5625,9.6875,0.3125,66.5000,31.9375,72.2500,844.845417,60.076667,2.906250,1436.088333,575.701667,419.24

In [26]:
full_dataset[full_dataset["label"]=="B"]

,mu11,mu12,mu13,mu14,mu15,mu16,mu17,mu18,mu19,mu110,max11,max12,max13,max14,max15,max16,max17,max18,max19,max110,med11,med12,med13,med14,med15,med16,med17,med18,med19,med110,grad11,grad12,grad13,grad14,grad15,grad16,grad17,grad18,grad19,grad110,mu21,mu22,mu23,mu24,mu25,mu26,mu27,mu28,mu29,mu210,max21,max22,max23,max24,max25,max26,max27,max28,max29,max210,med21,med22,med23,med24,med25,med26,med27,med28,med29,med210,grad21,grad22,grad23,grad24,grad25,grad26,grad27,grad28,grad29,grad210,label
35,214.143603,13.478603,-0.557304,720.261005,387.291618,267.620882,5.494044,1031.574338,641.857672,1151.346863,712.38625,44.34625,-0.04750,1672.24875,1059.24750,697.29000,14.38375,2469.85375,1624.42875,2245.81500,119.01125,7.72125,-0.67250,620.74875,277.74750,208.91500,4.75875,739.47875,494.17875,1370.06500,26.2500,1.9375,0.0625,84.1250,38.5000,26.3125,0.5000,143.0625,71.9375,128.9375,1375.199583,183.215417,4.190000,1712.501250,1072.880000,680.812500,24.917917,2534.187083,1825.741250,2296.090000,1501.38625,275.84625,6.57750,1794.12375,1256.87250,801.91500,27.88375,2575.97875,1871.42875,2347.69000,1459.01125,196.47125,4.82750,1735.18625,1117.49750,707.72750,26.82125,2544.91625,1838.92875,2310.37750,25.4375,2.2500,0.1875,10.0625,16.3125,9.1250,0.6250,10.0625,16.2500,8.0000,B
36,55.473088,2.788922,0.558382,245.704338,86.831985,74.354559,1.440319,354.303431,231.254510,519.241299,185.87750,7.98500,1.35250,847.35875,287.49375,233.08250,4.89375,1301.60000,746.85500,1447.36875,31.75250,1.48500,0.47750,113.73375,51.74375,50.70750,1.14375,135.60000,125.10500,326.61875,8.2500,0.3750,0.0625,33.1250,10.1250,7.7500,0.3125,55.0625,26.7500,60.0625,710.313333,44.034167,4.063333,1295.280417,475.674583,348.730000,9.220417,2079.948333,1318.166667,1820.671250,977.25250,73.98500,6.22750,1384.73375,547.24375,388.83250,10.64375,2192.22500,1439.60500,1897.49375,784.69000,45.92250,4.22750,1329.67125,489.55625,357.33250,10.01875,2154.35000,1413.41750,1850.43125,9.1250,0.5625,0.1875,26.2500,9.6250,6.3750,0.1875,44.9375,21.6250,22.3125,B
37,111.306691,5.782843,0.308480,439.338529,185.314240,145.734608,3.286544,672.947525,411.760098,830.197917,391.32875,19.92500,0.85750,1318.26500,592.83875,430.71500,8.55125,2035.01125,1190.51500,1918.40625,57.45375,2.92500,0.23250,228.64000,107.71375,97.84000,2.92625,314.01125,263.39000,744.90625,16.6250,0.8750,0.1875,55.3750,21.4375,14.7500,0.3750,91.8125,44.4375,91.9375,1084.097917,94.411667,3.902500,1547.605833,766.377083,523.486667,14.856250,2331.937917,1594.838333,2106.891250,1285.95375,149.42500,5.85750,1622.64000,882.83875,587.84000,16.92625,2382.76125,1661.14000,2161.53125,1213.76625,99.92500,4.23250,1565.26500,787.40125,535.21500,15.92625,2350.19875,1632.82750,2122.65625,16.6250,1.1875,0.1875,21.2500,16.5625,9.7500,0.3125,27.3125,22.7500,15.1250,B
38,212.767083,12.411054,-1.022451,726.727770,389.463799,281.032328,5.656863,1027.564828,648.094387,1149.136422,710.68375,43.65125,-0.14500,1678.55375,1065.84125,710.96125,14.62500,2469.49375,1597.25125,2220.58250,118.68375,6.65125,-1.14500,635.55375,273.09125,226.83625,5.00000,721.74375,534.00125,1386.95750,26.1875,1.8125,0.0625,87.5000,39.2500,27.1250,0.5625,145.9375,73.0000,133.0000,1368.170417,183.720417,3.512500,1716.630417,1091.854583,690.142917,25.093333,2533.082083,1777.053750,2270.709167,1493.30875,279.02625,6.10500,1791.80375,1259.96625,801.83625,28.12500,2571.24375,1818.00125,2317.58250,1453.68375,196.52625,4.10500,1737.42875,1131.34125,717.39875,26.87500,2542.49375,1788.87625,2283.52000,25.1875,2.2500,0.1875,9.4375,16.0625,8.3750,0.4375,9.8750,14.8750,7.2500,B
39,147.572010,7.615147,0.176348,553.480637,255.747623,185.916373,4.184020,806.602623,456.868309,930.402647,499.67250,25.87250,1.05625,1470.88750,760.06625,521.23500,10.63500,2215.29625,1300.09625,2022.41000,81.17250,3.99750,0.05625,362.63750,162.81625,131.86000,3.63500,444.04625,291.47125,926.41000,19.8125,1.3750,0.1875,63.8750,26.8125,18.2500,0.3750,105.8750,50.0000,99.0000,1205.738333,120.755000,4.492083,1625.184167

#### Save to csv file

In [27]:
full_dataset.to_csv("../dataset/data_2c40d2w.csv", index=False)